In [4]:
import numpy as np
import pandas as pd
import re
import nltk 
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import textstat as ts
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
#Probably won't use all of these, but I'll just import them all here anyway

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jrnoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking at some of the functions we wanted, I see that the length of the speech was one. textstat actually has that already - it is called `lexicon_count` (for number of words) or `sentence_count` (for number of sentences). This allows me to define some other simple functions:

In [9]:
def mreplace(text,list1,list2): #replaces strings in text more efficiently than built-in methods
    for i in range(len(list1)):
        text=text.replace(list1[i],list2[i])
    return text

def remover(text,removees): #removes a list of strings from text
    nothing = [ "" for i in removees ]
    return mreplace(text,removees,nothing)

def plaintxt(text,nospace=True): #removes all spaces, line breaks, and punctuation from text
    removees = ["\n",".",",","?","(",")","!","/",":",";","-","'",'"']
    if nospace: # If nospace is false, spaces will not be removed
        removees.append(" ")
    return remover(text,removees)

def sentlength(text): #average sentence length
    return ts.lexicon_count(text)/ts.sentence_count(text)

def wordlength(text): #average letters per word
    return len(plaintxt(text))/ts.lexicon_count(text)

def wordsyll(text): #average syllables per word
    return ts.syllable_count(text)/ts.lexicon_count(text)

We can also get the reading level from textstat. The best one to use is probably the `text_standard` function with `float_output=True`.

In [8]:
allSW = set(stopwords.words('english')) #set of all stop words
'a' in allSW 

True

In [10]:
def stopwordprop(text): #Proportion of stop words in text
    nopunc = plaintxt(text,nospace=False) #get rid of punctuations and line breaks
    words = nopunc.lower().split() #remove capitalization and split into individual words
    count = 0
    for word in words: #count the number of stop words in the text
        if word in allSW:
            count += 1 
    return count/len(words)

In [16]:
speeches = pd.read_csv("archive/presidential_speeches.csv")["Transcript"]
speeches=speeches.apply(str)
speeches.apply(sentlength)

0       59.750000
1      108.750000
2       40.380952
3       35.025000
4       29.208333
          ...    
987     17.755814
988     18.097222
989     11.971831
990     17.610092
991     12.823183
Name: Transcript, Length: 992, dtype: float64

In [17]:
speeches.apply(wordlength)

0      4.943515
1      4.751724
2      5.128538
3      4.895075
4      4.517832
         ...   
987    4.801572
988    4.722947
989    4.185647
990    4.936963
991    4.419641
Name: Transcript, Length: 992, dtype: float64

In [18]:
speeches.apply(stopwordprop)

0      0.544630
1      0.528736
2      0.534198
3      0.548111
4      0.543324
         ...   
987    0.460535
988    0.444530
989    0.516137
990    0.454262
991    0.500536
Name: Transcript, Length: 992, dtype: float64

Idea for making function for two & three word phrase count: 
Create a list of all words in plain text, as before. Then create a list of the form `[(concatenation of words i thru i+n-1) for i in range(word_count-n+1)]`, where n is the number of words in the phrases you are going for. Then use the vectorizor.

In [31]:
def phrases(text,n):
    nopunc = plaintxt(text,nospace=False) #probably want to add something here to deal with sentences ending, but
                                          # I'll do that later
    words = nopunc.lower().split()
    phraselist = [" ".join(words[i:i+n]) for i in range(len(words)-n+1)] #I might need to change " " to "" when I 
                                            # use this with the word vectorizer
    return phraselist

In [33]:
speech = speeches[0]
speech

'Fellow Citizens of the Senate and the House of Representatives: Among the vicissitudes incident to life, no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the fourteenth day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years: a retreat which was rendered every day more necessary as well as more dear to me, by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my Country called me, being sufficient to awaken in the wisest and most experienced of her citizens, a distrustful scrutiny into his qualification, could not

In [35]:
phrases(speech,2)

['fellow citizens',
 'citizens of',
 'of the',
 'the senate',
 'senate and',
 'and the',
 'the house',
 'house of',
 'of representatives',
 'representatives among',
 'among the',
 'the vicissitudes',
 'vicissitudes incident',
 'incident to',
 'to life',
 'life no',
 'no event',
 'event could',
 'could have',
 'have filled',
 'filled me',
 'me with',
 'with greater',
 'greater anxieties',
 'anxieties than',
 'than that',
 'that of',
 'of which',
 'which the',
 'the notification',
 'notification was',
 'was transmitted',
 'transmitted by',
 'by your',
 'your order',
 'order and',
 'and received',
 'received on',
 'on the',
 'the fourteenth',
 'fourteenth day',
 'day of',
 'of the',
 'the present',
 'present month',
 'month on',
 'on the',
 'the one',
 'one hand',
 'hand i',
 'i was',
 'was summoned',
 'summoned by',
 'by my',
 'my country',
 'country whose',
 'whose voice',
 'voice i',
 'i can',
 'can never',
 'never hear',
 'hear but',
 'but with',
 'with veneration',
 'veneration and',

In [19]:
def phrase_count(text,n,minnum=0):
    ngrams = phrases(text,n)
    unique_phrases = np.unique(np.array(ngrams))
    counter = {}
    for gram in unique_phrases:
        num = ngrams.count(gram)
        if num>=minnum: counter[gram] = num
    return counter